# Filtering and Aggregation

In this chapter we introduce the following query syntax:

* `WHERE` for filtering
* `AS` and other aliasing syntax

We will also introduce several boolean operators useful in expressions:

* `AND`, `OR`, `NOT`
* `=`, `<>`
* `>`, `>=`, `<`, `<=`

Finally, we will introduce several aggregate functions:

* `AVG`, `SUM`
* `MIN`, `MAX`
* `COUNT`


In [1]:
#| echo: false

import pandas as pd
import pyhive.sqlalchemy_presto

# always show every column
pd.set_option('display.max_columns', None)
# suppress a SQLAlchemy warning
pyhive.sqlalchemy_presto.PrestoDialect.supports_statement_cache = False

%load_ext sql
%config SqlMagic.autocommit = False
%config SqlMagic.displaycon = False
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False

%sql presto://localhost:8080/

In [2]:
#| echo: false
#| output: false

## just a reminder
%sql select * from listings limit 5

,listing_id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,reviews_per_month,access_date,country,state,city
0,2595,https://www.airbnb.com/rooms/2595,20220603182654,2022-06-04T00:00:00+00:00,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,"New York, United States",Midtown,Manhattan,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,NaN,1 bath,NaN,1.0,"[""Essentials"", ""Bathtub"", ""Extra pillows and b...",225.0,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,True,0,3,33,308,2022-06-04T00:00:00+00:00,48,0,0,2009-11-21T00:00:00+00:00,2019-11-04T00:00:00+00:00,4.70,4.72,4.62,4.76,4.79,4.86,4.41,,False,0.31,2022-06-03,united-states,ny,new-york-city
1,5121,https://www.airbnb.com/rooms/5121,20220603182654,2022-06-04T00:00:00+00:00,BlissArtsSpace!,One room available for rent in a 2 bedroom apt...,,https://a0.muscache.com/pictures/2090980c-b68e...,7356,,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,Private room in rental unit,Private room,2,NaN,,1.0,1.0,"[""Heating"", ""Long term stays allowed"", ""Kitche...",60.0,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,True,30,60,90,365,2022-06-04T00:00:00+00:00,50,0,0,2009-05-28T00:00:00+00:00,2019-12-02T00:00:00+00:00,4.52,4.22,4.09,4.91,4.91,4.47,4.52,,False,0.32,2022-06-03,united-states,ny,new-york-city
2,5136,https://www.airbnb.com/rooms/5136,20220603182654,2022-06-04T00:00:00+00:00,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,,https://a0.muscache.com/pictures/miso/Hosting-...,7378,,Sunset Park,Brooklyn,40.66265,-73.99454,Entire rental unit,Entire home/apt,4,NaN,1.5 baths,2.0,2.0,"[""Dryer"", ""Heating"", ""Hair dryer"", ""Carbon mon...",275.0,21,1125,21.0,21.0,1125.0,1125.0,21.0,1125.0,NaN,True,3,3,4,250,2022-06-04T00:00:00+00:00,2,1,0,2014-01-02T00:00:00+00:00,2021-08-08T00:00:00+00:00,5.00,5.00,5.00,5.00,5.00,4.50,5.00,,False,0.02,2022-06-03,united-states,ny,new-york-city
3,5178,https://www.airbnb.com/rooms/5178,20220603182654,2022-06-04T00:00:00+00:00,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",https://a0.muscache.com/pictures/12065/f070997...,8967,"New York, United States",Midtown,Manhattan,40.76457,-73.98317,Private room in rental unit,Private room,2,NaN,1 bath,1.0,1.0,"[""Conditioner"", ""Essentials"", ""Extra pillows a...",68.0,2,14,2.0,2.0,14.0,14.0,2.0,14.0,NaN,True,3,5,9,172,2022-06-04T00:00:00+00:00,536,62,2,2009-05-06T00:00:00+00:00,2022-05-09T00:00:00+00:00,4.23,4.24,3.75,4.66,4.44,4.87,4.39,,False,3.37,2022-06-03,united-states,ny,new-york-city
4,5203,https://www.airbnb.com/rooms/5203,20220603182654,2022-06-03T00:00:00+00:00,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",Our neighborhood is full of restaurants and ca...,https://a0.muscache.com/pictures/103776/b37157...,7490,"New York, United States",Upper West Side,Manhattan,40.80380,-73.96751,Private room in rental unit,Private room,1,NaN,1 shared bath,1.0,1.0,"[""Carbon monoxide alarm"", ""Heating"", ""Essentia...",75.0,2,14,2.0,2.0,14.0,14.0,2.

## `where`

So far we have "filtered" return data only in the most trivial sense: restricting the number of rows using `limit`:

In [3]:
%%sql

select
    listing_id,
    price,
    accommodates,
    neighbourhood_cleansed,
    city,
    state,
    country
from listings
limit 5

,listing_id,price,accommodates,neighbourhood_cleansed,city,state,country
0,2595,225.0,1,Midtown,new-york-city,ny,united-states
1,5121,60.0,2,Bedford-Stuyvesant,new-york-city,ny,united-states
2,5136,275.0,4,Sunset Park,new-york-city,ny,united-states
3,5178,68.0,2,Midtown,new-york-city,ny,united-states
4,5203,75.0,1,Upper West Side,new-york-city,ny,united-states


We can restrict our attention to rows matching specific criteria using the `where` clause, as in:

```sql
select
    [column specification]
from [source specification]
where [filter condition]
```

`[filter condition]` is a boolean expression that will look familiar to coders with any level of experience.  However, SQL does have some quirks that will take some getting used to:

* The equality comparison operator is `=`, **NOT** `==`
* The inequality comparison operator is `<>`

Let's take a look at some examples.  Here is another arbitrary set of listings, but this time from Paris:

In [4]:
%%sql

select
    listing_id,
    price,
    accommodates,
    neighbourhood_cleansed,
    city,
    state,
    country
from listings
where city = 'paris'
limit 5

,listing_id,price,accommodates,neighbourhood_cleansed,city,state,country
0,26660141,90.0,4,Buttes-Chaumont,paris,ile-de-france,france
1,25177338,59.0,2,Buttes-Chaumont,paris,ile-de-france,france
2,25218084,500.0,10,Batignolles-Monceau,paris,ile-de-france,france
3,23135649,65.0,2,Passy,paris,ile-de-france,france
4,23259887,51.0,2,Reuilly,paris,ile-de-france,france


Here are some listings that are not from New York City:

In [5]:
%%sql

select
    listing_id,
    price,
    accommodates,
    neighbourhood_cleansed,
    city,
    state,
    country
from listings
where city <> 'new-york-city'
limit 5

,listing_id,price,accommodates,neighbourhood_cleansed,city,state,country
0,23726706,88.0,2,IJburg - Zeeburgereiland,amsterdam,north-holland,the-netherlands
1,35815036,105.0,2,Noord-Oost,amsterdam,north-holland,the-netherlands
2,31553121,152.0,4,Noord-West,amsterdam,north-holland,the-netherlands
3,34745823,87.0,2,Gaasperdam - Driemond,amsterdam,north-holland,the-netherlands
4,44586947,160.0,4,Gaasperdam - Driemond,amsterdam,north-holland,the-netherlands


Here are some NYC listings that accommodate at least 4 guests:

In [6]:
%%sql

select
    listing_id,
    price,
    accommodates,
    neighbourhood_cleansed,
    city,
    state,
    country
from listings
where
    city = 'new-york-city'
    and accommodates >= 4
limit 5

,listing_id,price,accommodates,neighbourhood_cleansed,city,state,country
0,5136,275.0,4,Sunset Park,new-york-city,ny,united-states
1,7097,199.0,4,Fort Greene,new-york-city,ny,united-states
2,7801,395.0,4,Williamsburg,new-york-city,ny,united-states
3,8490,135.0,5,Bedford-Stuyvesant,new-york-city,ny,united-states
4,12937,140.0,4,Long Island City,new-york-city,ny,united-states


Above we introduced the boolean `and`.  Here is an extremely contrived, but equivalent, example that introduces both `or` and `not`:

In [7]:
%%sql

select
    listing_id,
    price,
    accommodates,
    neighbourhood_cleansed,
    city,
    state,
    country
from listings
where
    city = 'new-york-city'
    and (
        accommodates = 4
        or not (
            accommodates < 5
        )
    )
limit 5

,listing_id,price,accommodates,neighbourhood_cleansed,city,state,country
0,5136,275.0,4,Sunset Park,new-york-city,ny,united-states
1,7097,199.0,4,Fort Greene,new-york-city,ny,united-states
2,7801,395.0,4,Williamsburg,new-york-city,ny,united-states
3,8490,135.0,5,Bedford-Stuyvesant,new-york-city,ny,united-states
4,12937,140.0,4,Long Island City,new-york-city,ny,united-states


## `where` with `order by`

We saw in the previous chapter that we can learn about rows at the extremes by using `order by` and `limit`.  For example, we looked at hosts with the most listings.  Let's now find the Amsterdam-based host with the most listings:

In [8]:
%%sql

select
    host_id,
    name,
    since,
    city,
    calculated_listings_count
from hosts
where city = 'amsterdam'
order by
    calculated_listings_count desc
limit 5

,host_id,name,since,city,calculated_listings_count
0,203731852,SWEETS Hotel,2018-07-20T00:00:00+00:00,amsterdam,20
1,14574533,Hotel Not Hotel,2014-04-22T00:00:00+00:00,amsterdam,20
2,138369331,Peter,2017-07-03T00:00:00+00:00,amsterdam,20
3,67005410,Feliciano,2016-04-13T00:00:00+00:00,amsterdam,17
4,241644101,Wittenberg,2019-02-07T00:00:00+00:00,amsterdam,15


Returning to the listings table, let's find the cheapest option in NYC that accommodates at least 4 guests and tolerates stays as short as just 2 nights:

In [9]:
%%sql

select
    listing_id,
    price,
    accommodates,
    minimum_nights,
    neighbourhood_cleansed,
    city,
    state,
    country
from listings
where
    city = 'new-york-city'
    and accommodates >= 4
    and minimum_nights <= 2
order by
    price
limit 5

,listing_id,price,accommodates,minimum_nights,neighbourhood_cleansed,city,state,country
0,42065543,0.0,4,1,Midtown,new-york-city,ny,united-states
1,42065564,0.0,5,1,Financial District,new-york-city,ny,united-states
2,42279171,0.0,6,1,Chinatown,new-york-city,ny,united-states
3,41792753,0.0,6,1,Financial District,new-york-city,ny,united-states
4,41740615,0.0,4,1,Midtown,new-york-city,ny,united-states


That doesn't look right, does it?  These hosts are probably not inviting guests to stay for free, so let's spot check one of these and see if we can understand what's going on.

Since we're working in Jupyter, we can extract a `listing_id` from the previous cell's output like so:

In [10]:
listing_id = _.listing_id.iloc[0]
listing_id

42065543

Then we can pull all data for this listing:

In [11]:
%%sql

select *
from listings
where listing_id = {listing_id}

,listing_id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,reviews_per_month,access_date,country,state,city
0,42065543,https://www.airbnb.com/rooms/42065543,20220603182654,2022-06-03T00:00:00+00:00,Broadway Plaza Hotel,,,https://a0.muscache.com/pictures/miso/Hosting-...,307634016,,Midtown,Manhattan,40.74444,-73.9892,Room in boutique hotel,Hotel room,4,NaN,,NaN,NaN,"[""Bathtub"", ""Geneva Guild toiletries"", ""Bed sh...",0.0,1,365,1.0,1.0,1125.0,1125.0,1.0,1125.0,NaN,True,0,0,0,0,2022-06-03T00:00:00+00:00,316,163,5,2020-11-01T00:00:00+00:00,2022-05-25T00:00:00+00:00,4.66,4.72,4.69,4.87,4.86,4.89,4.61,,False,16.34,2022-06-03,united-states,ny,new-york-city


For good measure, let's also take a look at some recent reviews:

In [12]:
%%sql listing_review_comments <<

select review_date, comments
from reviews r
where listing_id = {listing_id}
order by review_id desc
limit 4

Returning data to local variable listing_review_comments


In [13]:
import textwrap

print('\n\n'.join(
    f'{r.review_date}:\n{textwrap.fill(r.comments)}'
    for r in listing_review_comments.itertuples()
))

2022-05-25T00:00:00:
Fantastic stay at this boutique hotel.  Fabulous location - so many
great choices for dining, convenient to the 28th Street subway stop.
The room was much larger than any others we've had in New York City -
and we've stayed at quite a few hotels.  Perfectly fine for our girls'
weekend! We were lucky with getting into our room early and stowing
our bags the day of departure to maximize time for fun.  Hotel staff
are friendly and were helpful with recommendations.  Would return and
highly recommend this hotel!

2022-05-23T00:00:00:
Great affordable upgraded downtown hotel with accommodating and
friendly staff.  Lots of great restaurants nearby and the neighborhood
is energized even into the later hours. Room had some normal street
noise and city lights coming in. Bathroom was clean and updated. Bed
was decently comfy. I would come back here, ok walk to lots of parks
and attractions.

2022-05-22T00:00:00:
Hammer Lage und freundliches Personal. Lediglich bei den Zimmer

So in this case we have what appears to be a legitimate listing, but it's at a boutique hotel.  Far to the right in the `select *`, we see that `instant_bookable` is false.  Maybe you have to initiate the booking process before the price is revealed.

Anyway, we can add another condition `price > 0` to find some more plausible, but still cheap, prices:

In [14]:
%%sql

select
    listing_id,
    price,
    accommodates,
    minimum_nights,
    neighbourhood_cleansed,
    city,
    state,
    country
from listings
where
    city = 'new-york-city'
    and accommodates >= 4
    and minimum_nights <= 2
    and price > 0
order by
    price
limit 5

,listing_id,price,accommodates,minimum_nights,neighbourhood_cleansed,city,state,country
0,28607153,35.0,4,1,Jackson Heights,new-york-city,ny,united-states
1,45565991,40.0,4,1,East Flatbush,new-york-city,ny,united-states
2,51184699,40.0,4,1,Inwood,new-york-city,ny,united-states
3,49535021,42.0,4,1,East Elmhurst,new-york-city,ny,united-states
4,36082249,45.0,4,1,Borough Park,new-york-city,ny,united-states


## Basic Aggregates

Sometimes filtering with `where` and sorting with `order by` is exactly what is needed in order to learn about data in context at the extremes.  But sometimes you just want to know what the extreme values are.

This brings us to _aggregation_ and _aggregates_.  An aggregate, or aggregate function, is a calculation that operates on multiple values and returns a scalar value.  Aggregation is the process of preparing to compute, and ultimately computing, one or more aggregate functions.

### `min()` and `max()`

Above we were interested in low prices.  We can find the lowest price with `min()`:

In [15]:
%%sql

select
    min(price)
from listings
where
    city = 'new-york-city'
    and accommodates >= 4
    and minimum_nights <= 2
    and price > 0

,"MIN(""listings"".""price"")"
0,35.0


Conversely, using `max()` we can look for the most expensive room that only accommodates a single guest:

In [16]:
%%sql

select
    max(price)
from listings
where
    city = 'new-york-city'
    and accommodates = 1

,"MAX(""listings"".""price"")"
0,10000.0


### `count()`

One of the most useful aggregates is `count()`:

In [17]:
%%sql

select
    count(1)
from listings
where city = 'new-york-city'

,COUNT(1)
0,37410


Note that `count()` counts non-null elements, **not** the number of rows!  For example:

In [18]:
%%sql

select
    count(1),
    count(beds),
    count(bedrooms)
from listings
where city = 'new-york-city'
    and accommodates >= 4

,COUNT(1),"COUNT(""listings"".""beds"")","COUNT(""listings"".""bedrooms"")"
0,11026,10914,10474


Above, `count(beds)` differs from `count(bedrooms)` because "bedrooms" has more nulls than "beds".

Note that `count(1)` is somewhat idiomatic, but any non-null literal will have the same effect, which is to simply count rows:

In [19]:
%%sql

select
    count(1),
    count(2),
    count('unnecessary input here')
from listings
where city = 'new-york-city'
    and accommodates >= 4

,COUNT(1),COUNT(2),COUNT('unnecessary input here')
0,11026,11026,11026


### `sum()` and `avg()`

Two more extremely useful aggregates are `sum()` and `avg()`:

In [20]:
%%sql

select
    count(1),
    sum(beds),
    avg(beds)
from listings
where city = 'new-york-city'
    and accommodates >= 4

,COUNT(1),"SUM(""listings"".""beds"")","AVG(""listings"".""beds"")"
0,11026,29610.0,2.713029


:::{.callout-note}

Yes, in SQL `avg` is the way we spell "mean" or "average" – don't worry, you get used to it.

:::

## Aliasing

Aggregate result default column names are implementation-dependent.  We saw above that dask-sql assigns default names that reflect the expression being evaluated – for example, `avg(beds)` $\Rightarrow$ `AVG("listings"."beds")`.

This default is ugly enough to motivate an alternative, but also explicit enough to give us a hint about SQL's column naming syntax.

In most (all?) implementations, single quotes are used for strings while double quotes are used for schema/table/column names.  The above query could be written as:

In [21]:
%%sql

select
    count(1),
    sum("listings"."beds"),
    avg("listings"."beds")
from "listings"
where "city" = 'new-york-city'
    and "accommodates" >= 4

,COUNT(1),"SUM(""listings"".""beds"")","AVG(""listings"".""beds"")"
0,11026,29610.0,2.713029


Aliases are specified using either of the following syntaxes:

* `<name> <alias>`
* `<name> AS <alias>`

For well-behaved names (identifiers that are not reserved words) the double-quotes are optional.  The `AS` is also optional.  So the most concise way to name our columns would be:

In [22]:
%%sql

select
    count(1) num_listings,
    sum(beds) total_beds,
    avg(beds) mean_beds
from listings
where city = 'new-york-city'
    and accommodates >= 4

,num_listings,total_beds,mean_beds
0,11026,29610.0,2.713029


Here's an example gratuitously inconsistent style that shows some of the various valid syntactic combinations:

In [23]:
%%sql

select
    count(listing_id) as "num_listings",
    sum(beds) as total_beds,
    avg(l.beds) as "mean_beds"
from listings l
where city = 'new-york-city'
    and accommodates >= 4

,num_listings,total_beds,mean_beds
0,11026,29610.0,2.713029


## Teaser: aggregates with variable filters

Above we found some "number of beds" aggregates for New York City.  In this environment, embedded in JupyterLab, we can run similar calculations for all cities in the dataset:

In [24]:
cities = %sql select distinct city from listings
cities = cities.city.values
cities

array(['new-york-city', 'amsterdam', 'paris'], dtype=object)

In [25]:
for city in cities:
    x = %sql select '{city}' city, avg(beds) mean_beds from listings where city = '{city}' and accommodates >= 4
    display(x)

,city,mean_beds
0,new-york-city,2.713029


,city,mean_beds
0,amsterdam,3.152174


,city,mean_beds
0,paris,2.664234


Here we've introduced some new JupyterLab-specific SQL magic syntax: `%sql` with a single `%` can be followed by a single-line SQL query as an expression which can be assigned into a variable.

More importantly, we've performed our first analysis consisting of an aggregate computed on some numerical column, for each possible value of some other categorical column.  In the next chapter we'll explore SQL's dedicated syntax for this type of analysis.

## Exercises

**TODO.**